In [2]:
import os
os.environ['HF_HOME'] = '/workspace/cache/huggingface/'
os.environ['HF_HOME'] = '/workspace/cache/huggingface/'
os.chdir('/workspace/FutureGPT2/src/')

import datasets
from torch.utils.data import DataLoader
from datasets import load_dataset
from data.mmap_dataset import MMapIndexedDataset
from tqdm.notebook import tqdm
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer


%load_ext autoreload
%autoreload 2

In [3]:
PILE_PATH = '/workspace/corpus/pythia_pile_idxmaps/pile_0.87_deduped_text_document.bin'
pile = MMapIndexedDataset(PILE_PATH, skip_warmup=True)

    reading sizes...
    reading pointers...
    reading document index...
    creating numpy buffer of mmap...
    creating memory view of numpy buffer...


In [4]:
SEQ_LEN = 64
SIZE = 20_000_000

In [5]:
data_list = []
for x in tqdm(pile):
    if len(x) >= SEQ_LEN:
        data_list.append({'input_ids': x[:SEQ_LEN]})
    if len(data_list) > SIZE:
        break

  0%|          | 0/134318121 [00:00<?, ?it/s]

In [7]:
dataset = datasets.Dataset.from_list(data_list)

In [8]:
train_test = dataset.train_test_split(test_size=0.01)

In [10]:
filename = f'/workspace/corpus/the_pile/pile_PYTHIA_{SEQ_LEN}tokens_20M'

In [11]:
train_test.save_to_disk(filename)

Saving the dataset (0/6 shards):   0%|          | 0/19800000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200001 [00:00<?, ? examples/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-14m')
Token = {v: k for k, v in tokenizer.get_vocab().items()}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
print(''.join(Token[x] for x in dataset[0]['input_ids']))

ItĠisĠdone,ĠandĠsubmitted.ĠYouĠcanĠplayĠâĢľSurvivalĠofĠtheĠTastiestâĢĿĠonĠAndroid,ĠandĠonĠtheĠweb.ĠPlayingĠonĠtheĠwebĠworks,ĠbutĠyouĠhaveĠtoĠsimulateĠmulti-touchĠforĠtableĠmovingĠandĠthatĠcanĠbeĠaĠbitĠconfusing.ĊĊThereâĢĻsĠaĠlotĠIâĢĻdĠlikeĠto
